# References

[NeRF Benchmark Datasets](http://cseweb.ucsd.edu/~viscomp/projects/LF/papers/ECCV20/nerf/nerf_example_data.zip)

[NeRF Pytorch Implementation](https://github.com/yenchenlin/nerf-pytorch)

[Original COLMAP Implementation](https://github.com/colmap/colmap)

[Google Colab COLMAP Implementation](https://github.com/Abbsalehi/colmap)

[LLFF Pose Generator](https://github.com/Fyusion/LLFF)

---

# 1. Data Preparation: Benchmark or Custom Images

This section guides you through setting up your image data. You can either start with a provided benchmark dataset for quick testing or use your own captured images.

---

## Option 1: Using the Fern Benchmark Dataset (Recommended for First-Time Users)

This option allows you to quickly test the notebook with a pre-processed, downsampled dataset.

**Instructions:**

1.  **Download Benchmark Data:**
    *   Download the NeRF example data from: [http://cseweb.ucsd.edu/~viscomp/projects/LF/papers/ECCV20/nerf/nerf_example_data.zip](http://cseweb.ucsd.edu/~viscomp/projects/LF/papers/ECCV20/nerf/nerf_example_data.zip)

2.  **Extract and Place `fern` Folder:**
    *   Extract the downloaded ZIP file.
    *   Locate the `fern` folder within the `nerf_llff_data` folder.
    *   Move this `fern` folder into your Google Drive at the following path:
        `drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/data/`

3.  **Prepare the `fern` Dataset:**
    *   Navigate into the `drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/data/fern/` folder.
    *   Delete all files and folders **except** for the `images_4` folder and the `sparse` folder.
    *   Ensure the `sparse` folder is **empty**. If it exists and contains files, delete all files inside it.
    *   Rename the `images_4` folder to `images`.
        *   *Note: We use `images_4` (which are images downsampled by a factor of 4) because the original full-resolution images can be too large for timely processing in a free Google Colab session.*

    *After these steps, your `fern` directory should look like this:*
    ```
    NeRF-Training-in-Google-Colab/
    └── nerf_pytorch/
        └── data/
            └── fern/
                ├── images/       
                │   ├── image000.png
                │   ├── image001.png
                │   └── ...
                └── sparse/       (should be an empty folder)
    ```

---

## Option 2: Using Your Own Custom Images

If you've successfully run the benchmark or wish to use your own data:

**Instructions:**

1.  **Image Capture:**
    *   Acquire approximately 20-40 photographs of your target object.
    *   Ensure consistent and adequate lighting conditions.
    *   Vary viewpoints around the object, maintaining good overlap between images.
    *   Convert images to `.png` format.

2.  **Directory Organization:**
    *   Create a main folder for your subject (e.g., `my_truck`).
    *   Inside this subject folder, create a subfolder named `images`.
    *   Place all captured photographs into this `images` subfolder.
    *   **Crucially, create an empty subfolder named `sparse` inside your subject folder.** COLMAP will write its output here.
    *   Move your main subject folder (e.g., `my_truck`) into your Google Drive at:
        `drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/data/`

    *Expected Structure for custom data:*
    ```
    NeRF-Training-in-Google-Colab/
    └── nerf_pytorch/
        └── data/
            └── your_subject_name/
                ├── images/
                │   ├── image001.png  
                │   ├── image002.png
                │   └── ...
                └── sparse/           (should be an empty folder)
    ```

---

## Configure `basedir` Path

The `basedir` variable in the next code cell tells the notebook where to find your image data.

*   **Set `basedir` to the path of your chosen subject's main folder** (e.g., `fern` or `my_truck`).
*   **Crucially, do not include `/images` or `/sparse` in the `basedir` path itself.**

In [1]:
basedir = "/content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/data/fern"

---

# 2. Camera Pose Estimation with COLMAP

This section utilizes COLMAP, a Structure-from-Motion (SfM) and Multi-View Stereo (MVS) software, to estimate the camera poses (position and orientation) for each image in your dataset. These poses are essential for NeRF training.

**Note on COLMAP in Google Colab:**

Standard COLMAP installations include GUI components that are incompatible with the Google Colab environment. To address this, we use a specialized build of COLMAP that excludes these UI elements, enabling its use within this notebook.

Special thanks to [Abbas Salehi](https://github.com/Abbsalehi) for providing the necessary modifications to build COLMAP from source without GUI dependencies.

### Install the Required Packages

In [2]:
!sudo apt-get install -y git
!sudo apt-get install -y cmake
!sudo apt-get install -y ninja-build
!sudo apt-get install -y build-essential
!sudo apt-get install -y libboost-program-options-dev
!sudo apt-get install -y libboost-filesystem-dev
!sudo apt-get install -y libboost-graph-dev
!sudo apt-get install -y libboost-system-dev
!sudo apt-get install -y libeigen3-dev
!sudo apt-get install -y libflann-dev
!sudo apt-get install -y libfreeimage-dev
!sudo apt-get install -y libmetis-dev
!sudo apt-get install -y libgoogle-glog-dev
!sudo apt-get install -y libgtest-dev
!sudo apt-get install -y libsqlite3-dev
!sudo apt-get install -y libglew-dev
!sudo apt-get install -y qtbase5-dev
!sudo apt-get install -y libqt5opengl5-dev
!sudo apt-get install -y libcgal-dev
!sudo apt-get install -y libceres-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  ninja-build
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 111 kB of archives.
After this operation, 358 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ninja-build amd64 1.10.1-1 [111 kB]
Fetched 111 kB in 1s (95.8 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the

### Clone Colmap Repository

In [3]:
!pwd
!git clone https://github.com/colmap/colmap.git
%cd colmap

/content
Cloning into 'colmap'...
remote: Enumerating objects: 27774, done.
remote: Counting objects: 100% (327/327), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 27774 (delta 235), reused 173 (delta 140), pack-reused 27447 (from 3)
Receiving objects: 100% (27774/27774), 70.95 MiB | 17.09 MiB/s, done.
Resolving deltas: 100% (21422/21422), done.
/content/colmap


### Build and Install Colmap From Source (May Take Up to 30 Minutes 😵)


In [4]:
!pwd
!mkdir build
%cd build
!cmake -GNinja ..
!ninja
!sudo ninja install

/content/colmap
/content/colmap/build
-- Enabling LSD support
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found FreeImage
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libfreeimage.so
-- Found FLANN
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libflann.so
-- Found LZ4
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/liblz4.so
-- Found Metis
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libmetis.so
-- Found Glog
--   Target : glog:

### Allow access to Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Detect and extract features from images for matching

In [6]:
!colmap feature_extractor \
    --database_path "{basedir}/database.db" \
    --image_path "{basedir}/images" \
    --ImageReader.single_camera 1

I0521 23:51:01.995775 12054 misc.cc:44] 
Feature extraction
I0521 23:51:01.996259 12057 sift.cc:721] Creating SIFT GPU feature extractor
I0521 23:51:03.804358 12058 feature_extraction.cc:259] Processed file [1/20]
I0521 23:51:03.804389 12058 feature_extraction.cc:262]   Name:            image000.png
I0521 23:51:03.804396 12058 feature_extraction.cc:271]   Dimensions:      1008 x 756
I0521 23:51:03.804401 12058 feature_extraction.cc:274]   Camera:          #1 - SIMPLE_RADIAL
I0521 23:51:03.804409 12058 feature_extraction.cc:277]   Focal Length:    1209.60px
I0521 23:51:03.804422 12058 feature_extraction.cc:281]   Features:        6231
I0521 23:51:05.471024 12058 feature_extraction.cc:259] Processed file [2/20]
I0521 23:51:05.471055 12058 feature_extraction.cc:262]   Name:            image001.png
I0521 23:51:05.471060 12058 feature_extraction.cc:271]   Dimensions:      1008 x 756
I0521 23:51:05.471064 12058 feature_extraction.cc:274]   Camera:          #1 - SIMPLE_RADIAL
I0521 23:51:05.4

### Match Features Between All Pairs of Images

In [7]:
!colmap exhaustive_matcher\
    --database_path "{basedir}/database.db"

I0521 23:51:37.420059 12228 misc.cc:44] 
Feature matching
I0521 23:51:37.421411 12229 sift.cc:1426] Creating SIFT GPU feature matcher
I0521 23:51:37.515944 12228 pairing.cc:168] Generating exhaustive image pairs...
I0521 23:51:37.515980 12228 pairing.cc:201] Matching block [1/1, 1/1]
I0521 23:51:38.789073 12228 feature_matching.cc:46] in 1.273s
I0521 23:51:38.842011 12228 timer.cc:91] Elapsed time: 0.024 [minutes]


### Reconstruct Camera Poses and Scene Geometry

This step executes the COLMAP pipeline to perform feature extraction, matching, and sparse 3D reconstruction. The output of this process, including the estimated camera poses and a sparse point cloud of the scene, will be saved to a new `sparse` directory.

In [8]:
!colmap mapper \
    --database_path "{basedir}/database.db" \
    --image_path "{basedir}/images" \
    --output_path "{basedir}/sparse" \
    --Mapper.num_threads 16 \
    --Mapper.init_min_tri_angle 4 \
    --Mapper.multiple_models 0 \
    --Mapper.extract_colors 0

I0521 23:51:44.493814 12263 incremental_pipeline.cc:253] Loading database
I0521 23:51:44.532851 12263 database_cache.cc:69] Loading rigs...
I0521 23:51:44.532977 12263 database_cache.cc:79]  1 in 0.000s
I0521 23:51:44.532994 12263 database_cache.cc:87] Loading cameras...
I0521 23:51:44.533013 12263 database_cache.cc:105]  1 in 0.000s
I0521 23:51:44.533022 12263 database_cache.cc:113] Loading frames...
I0521 23:51:44.533165 12263 database_cache.cc:130]  20 in 0.000s
I0521 23:51:44.533174 12263 database_cache.cc:138] Loading matches...
I0521 23:51:44.537796 12263 database_cache.cc:143]  190 in 0.005s
I0521 23:51:44.537820 12263 database_cache.cc:159] Loading images...
I0521 23:51:44.560583 12263 database_cache.cc:241]  20 in 0.023s (connected 20)
I0521 23:51:44.560612 12263 database_cache.cc:252] Building correspondence graph...
I0521 23:51:44.581911 12263 database_cache.cc:279]  in 0.021s (ignored 0)
I0521 23:51:44.581966 12263 timer.cc:91] Elapsed time: 0.001 [minutes]
I0521 23:51:44.5

### Process and Save COLMAP Results

The following cell uses utility functions, adapted from the [LLFF (Local Light Field Fusion) project](https://github.com/Fyusion/LLFF/blob/master/llff/poses/pose_utils.py), to process the output generated by COLMAP in the previous step.

Specifically, it will:
1.  Read the camera parameters (`cameras.bin`), image information (`images.bin`), and 3D points (`points3D.bin`) from the `sparse/0/` directory.
2.  Convert this data into a format suitable for NeRF training, including camera-to-world transformation matrices and depth bounds.
3.  Save these processed poses and bounds into a file named `poses_bounds.npy` directly within your subject's `basedir`. This `.npy` file is a critical input for the NeRF model.

In [9]:
import sys
sys.path.append('/content/drive/My Drive/NeRF-Training-in-Google-Colab/LLFF')
from llff.poses.pose_utils import load_colmap_data, save_poses

In [10]:
poses, pts3d, perm = load_colmap_data(basedir)
save_poses(basedir, poses, pts3d, perm)

Cameras 5
Images # 20
Points (6243, 3) Visibility (6243, 20)
Depth stats 7.840506978106666 88.55682961307406 29.856545739277003


---

# 3. Train


### Import NeRF Training and Rendering Core Logic

The following cell imports the primary `train` function and associated utilities necessary for training the NeRF model and rendering novel views. This code is a modified version of the original PyTorch NeRF implementation found [here](https://github.com/yenchenlin/nerf-pytorch/blob/master/run_nerf.py). Modifications have been made to adapt it for use within this Google Colab notebook environment, such as handling arguments and device initialization.

In [11]:
import sys
import argparse
sys.path.append('/content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch')
from run_nerf import train, generate_render

### Training Arguments Description

- **basedir**: The directory where checkpoints and logs will be stored.
- **datadir**: The input data directory containing the dataset to be trained on.
- **config**: The path to the configuration file containing training parameters and settings.

- **no_reload**: If `True`, prevents reloading weights from a saved checkpoint. Initial value was `False`.
- **ft_path**: Path to a specific weights tar file to reload for the coarse network.

- **dataset_type**: Type of dataset to be used. Options include `llff`, `blender`, and `deepvoxels`. Initial value was `llff`.
- **expname**: A name for the experiment.

- **N_iters**: Number of iterations (training steps). Initial value was `200000`.
- **i_img**: Frequency for tensorboard image logging. Set to an arbitrarily high number to disable. Initial value was `500`.
- **i_print**: Frequency of console printouts and metric logging. Initial value was `100`.
- **i_weights**: Frequency of saving weight checkpoints. Initial value was `10000`.

- **half_res**: If `True`, loads Blender synthetic data at 400x400 resolution instead of 800x800. Initial value was `False`.


In [12]:
train_args = argparse.Namespace(
        basedir='/content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/logs',
        datadir='/content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/data/fern',
        config='/content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/configs/fern.txt', #fern config can be used for most real-life scenes

        no_reload=True,
        ft_path= '',

        dataset_type='llff',
        expname='Fern-First-Test',

        N_iters=3001,
        i_img=999999999,
        i_print=100,
        i_weights= 1000,

        half_res=False,
    )

### Train Model

In [13]:
train(train_args)

/usr/local/lib/python3.11/dist-packages/torch/__init__.py:1236: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


Loaded image data (756, 1008, 3, 20) [ 756.         1008.          825.86310696]
Loaded /content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/data/fern 15.54141522130248 77.18885228064539
recentered (3, 5)
[[ 1.0000000e+00 -1.8826084e-10  4.4952766e-09  1.4901161e-09]
 [ 1.8826082e-10  1.0000000e+00  2.0603350e-09 -4.4703485e-09]
 [-4.4952766e-09 -2.0603350e-09  1.0000000e+00  1.8626451e-09]]
Data:
(20, 3, 5) (20, 756, 1008, 3) (20, 2)
HOLDOUT view is 12
Loaded llff (20, 756, 1008, 3) (60, 3, 5) [ 756.     1008.      825.8631] /content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/data/fern
Auto LLFF holdout, 8
DEFINING BOUNDS
NEAR FAR 0.0 1.0
Found ckpts ['']
Begin
TRAIN views are [ 1  2  3  4  5  6  7  9 10 11 12 13 14 15 17 18 19]
TEST views are [ 0  8 16]
VAL views are [ 0  8 16]


  0%|          | 0/3001 [00:00<?, ?it/s]

[Config] Center cropping of size 378 x 504 is enabled until iter 500


/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  3%|▎         | 100/3001 [00:34<16:47,  2.88it/s]

...

 97%|█████████▋| 2900/3001 [17:40<00:38,  2.64it/s]

[TRAIN] Iter: 2900 Loss: 0.023289665579795837  PSNR: 19.266942977905273


100%|█████████▉| 3000/3001 [18:17<00:00,  2.51it/s]

Saved checkpoints at /content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/logs/Fern-First-Test/003000.tar
[TRAIN] Iter: 3000 Loss: 0.01626673713326454  PSNR: 21.01634407043457


100%|██████████| 3001/3001 [18:17<00:00,  2.73it/s]


# 4. Render Images / Video



### Rendering Arguments Description

- **basedir**: The directory where checkpoints and logs will be stored.
- **datadir**: The input data directory containing the dataset to be rendered.
- **config**: The path to the configuration file containing rendering parameters and settings.
- **ft_path**: Path to a specific weights tar file to reload for the coarse network.

- **dataset_type**: Type of dataset to be used. Options include `llff`, `blender`, and `deepvoxels`. Initial value was `llff`.
- **expname**: A name for the experiment.

- **half_res**: If `True`, loads Blender synthetic data at 400x400 resolution instead of 800x800. Initial value was `False`.
- **num_frames**: Number of frames to generate. Initial value was `40`.




In [19]:
render_args = argparse.Namespace(
        basedir='/content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/logs',
        datadir='/content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/data/fern',
        config='/content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/configs/fern.txt',
        ft_path= '/content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/logs/Fern-First-Test/003000.tar',

        dataset_type='llff',
        expname='Fern-First-Test',

        half_res=False,
        num_frames = 10,
    )

### Generate Render

In [20]:
generate_render(render_args)

Loaded image data (756, 1008, 3, 20) [ 756.         1008.          103.23288837]
Loaded /content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/data/fern 15.54141522130248 77.18885228064539
recentered (3, 5)
[[ 1.0000000e+00 -1.8826084e-10  4.4952766e-09  1.4901161e-09]
 [ 1.8826082e-10  1.0000000e+00  2.0603350e-09 -4.4703485e-09]
 [-4.4952766e-09 -2.0603350e-09  1.0000000e+00  1.8626451e-09]]
Data:
(20, 3, 5) (20, 756, 1008, 3) (20, 2)
HOLDOUT view is 12
Loaded llff (20, 756, 1008, 3) (10, 3, 5) [ 756.      1008.       103.23289] /content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/data/fern
Auto LLFF holdout, 8
DEFINING BOUNDS
NEAR FAR 0.0 1.0
Found ckpts ['/content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/logs/Fern-First-Test/003000.tar']
Reloading from /content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/logs/Fern-First-Test/003000.tar
RENDER ONLY
test poses shape torch.Size([10, 3, 5])


  0%|          | 0/10 [00:00<?, ?it/s]

0 0.0019609928131103516
torch.Size([756, 1008, 3]) torch.Size([756, 1008])


 10%|█         | 1/10 [01:52<16:54, 112.75s/it]

1 112.75377678871155


...

8 112.38812899589539


 90%|█████████ | 9/10 [16:52<01:52, 112.47s/it]

9 112.4190182685852


100%|██████████| 10/10 [18:45<00:00, 112.52s/it]


Done rendering /content/drive/My Drive/NeRF-Training-in-Google-Colab/nerf_pytorch/logs/Fern-First-Test/renderonly_path_002999
